In [1]:
import plotly 
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
print(cf.__version__)
# Configure cufflings 
cf.set_config_file(offline=False, world_readable=True, theme='pearl')

0.12.1


In [2]:
import nltk
# nltk.download()
# !pip install autocorrect
from autocorrect import spell

In [3]:
import pandas as pd
import numpy as np
import gzip

In [64]:
# For monitoring duration of pandas processes
from tqdm import tqdm, tqdm_pandas

# To avoid RuntimeError: Set changed size during iteration
tqdm.monitor_interval = 0

# Register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
# (can use `tqdm_gui`, `tqdm_notebook`, optional kwargs, etc.)
tqdm.pandas(desc="Progress:")

# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
# can also groupby:
# df.groupby(0).progress_apply(lambda x: x**2)

### Data Loading

In [8]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

In [9]:
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [10]:
df = getDF('/Users/falehalrashidi/Downloads/reviews_Books_5.json.gz')

In [11]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A10000012B7CGYKOMPQ4L,000100039X,Adam,"[0, 0]",Spiritually and mentally inspiring! A book tha...,5.0,Wonderful!,1355616000,"12 16, 2012"
1,A2S166WSCFIFP5,000100039X,"adead_poet@hotmail.com ""adead_poet@hotmail.com""","[0, 2]",This is one my must have books. It is a master...,5.0,close to god,1071100800,"12 11, 2003"
2,A1BM81XB4QHOA3,000100039X,"Ahoro Blethends ""Seriously""","[0, 0]",This book provides a reflection that you can a...,5.0,Must Read for Life Afficianados,1390003200,"01 18, 2014"
3,A1MOSTXNIO5MPJ,000100039X,Alan Krug,"[0, 0]",I first read THE PROPHET in college back in th...,5.0,Timeless for every good and bad time in your l...,1317081600,"09 27, 2011"
4,A2XQ5LZHTD4AFT,000100039X,Alaturka,"[7, 9]",A timeless classic. It is a very demanding an...,5.0,A Modern Rumi,1033948800,"10 7, 2002"


In [12]:
df1 = df[['reviewerID','asin','reviewText','helpful']]

In [35]:
df1.head()

,reviewerID,asin,reviewText,helpful
0,A10000012B7CGYKOMPQ4L,000100039X,Spiritually and mentally inspiring! A book tha...,"[0, 0]"
1,A2S166WSCFIFP5,000100039X,This is one my must have books. It is a master...,"[0, 2]"
2,A1BM81XB4QHOA3,000100039X,This book provides a reflection that you can a...,"[0, 0]"
3,A1MOSTXNIO5MPJ,000100039X,I first read THE PROPHET in college back in th...,"[0, 0]"
4,A2XQ5LZHTD4AFT,000100039X,A timeless classic. It is a very demanding an...,"[7, 9]"


In [36]:
len(df1)

8898041

In [65]:
# Create new Column for the denominator
df2 = df1.assign(denom = df1['helpful'].progress_apply(lambda enum_denom:enum_denom[1]))





Progress::   0%|          | 0/8898041 [00:00<?, ?it/s]



Progress::   0%|          | 1/8898041 [00:00<644:57:01,  3.83it/s]



Progress::   1%|▏         | 117955/8898041 [00:00<445:28:52,  5.47it/s]



Progress::   3%|▎         | 252254/8898041 [00:00<307:04:03,  7.82it/s]



Progress::   5%|▍         | 402602/8898041 [00:00<211:12:35, 11.17it/s]



Progress::   6%|▌         | 555081/8898041 [00:00<145:11:37, 15.96it/s]



Progress::   8%|▊         | 709561/8898041 [00:00<99:45:14, 22.80it/s] 



Progress::  10%|▉         | 868508/8898041 [00:00<68:28:22, 32.57it/s]



Progress::  12%|█▏        | 1027768/8898041 [00:00<46:58:50, 46.53it/s]



Progress::  13%|█▎        | 1192164/8898041 [00:01<32:11:59, 66.48it/s]



Progress::  15%|█▌        | 1355747/8898041 [00:01<22:03:42, 94.96it/s]



Progress::  17%|█▋        | 1519446/8898041 [00:01<15:06:30, 135.66it/s]



Progress::  19%|█▉        | 1684676/8898041 [00:01<10:20:22, 193.79it/s]



Progress::  21%|██        | 1852449/889804

In [66]:
df2.head()

,reviewerID,asin,reviewText,helpful,denom
0,A10000012B7CGYKOMPQ4L,000100039X,Spiritually and mentally inspiring! A book tha...,"[0, 0]",0
1,A2S166WSCFIFP5,000100039X,This is one my must have books. It is a master...,"[0, 2]",2
2,A1BM81XB4QHOA3,000100039X,This book provides a reflection that you can a...,"[0, 0]",0
3,A1MOSTXNIO5MPJ,000100039X,I first read THE PROPHET in college back in th...,"[0, 0]",0
4,A2XQ5LZHTD4AFT,000100039X,A timeless classic. It is a very demanding an...,"[7, 9]",9


In [67]:
# Create a Uniquekey Column
df3 = df2.assign(uniqueKey = df['reviewerID'].str.cat(df['asin'].values.astype(str), sep='##'))

In [68]:
df3.head()

,reviewerID,asin,reviewText,helpful,denom,uniqueKey
0,A10000012B7CGYKOMPQ4L,000100039X,Spiritually and mentally inspiring! A book tha...,"[0, 0]",0,A10000012B7CGYKOMPQ4L##000100039X
1,A2S166WSCFIFP5,000100039X,This is one my must have books. It is a master...,"[0, 2]",2,A2S166WSCFIFP5##000100039X
2,A1BM81XB4QHOA3,000100039X,This book provides a reflection that you can a...,"[0, 0]",0,A1BM81XB4QHOA3##000100039X
3,A1MOSTXNIO5MPJ,000100039X,I first read THE PROPHET in college back in th...,"[0, 0]",0,A1MOSTXNIO5MPJ##000100039X
4,A2XQ5LZHTD4AFT,000100039X,A timeless classic. It is a very demanding an...,"[7, 9]",9,A2XQ5LZHTD4AFT##000100039X


In [69]:
# Keep only the columns necessary for the normalisation
df4 = df3[['uniqueKey', 'reviewText']]
df4.head()

,uniqueKey,reviewText
0,A10000012B7CGYKOMPQ4L##000100039X,Spiritually and mentally inspiring! A book tha...
1,A2S166WSCFIFP5##000100039X,This is one my must have books. It is a master...
2,A1BM81XB4QHOA3##000100039X,This book provides a reflection that you can a...
3,A1MOSTXNIO5MPJ##000100039X,I first read THE PROPHET in college back in th...
4,A2XQ5LZHTD4AFT##000100039X,A timeless classic. It is a very demanding an...


The next step was necessary due to weird keyErrors that followed after trying to process the reviewText as a `pandas.DataFrame` and not as `pandas.Series`. After experimenting with both, I found that `pandas.Series.apply` is faster than `pandas.DataFrame.apply` and so I will hence work with `pandas.Series`. 

The assumption I require to make at this point before I follow is that `pandas` will not change the index of the reviews as those are being processed by my code and that in the end of my processing I will be able to re-associate those reviews with their **uniqueKey**. 

In [70]:
uniqueKey_series_df = df4[['uniqueKey']]
uniqueKey_series_df.head()

,uniqueKey
0,A10000012B7CGYKOMPQ4L##000100039X
1,A2S166WSCFIFP5##000100039X
2,A1BM81XB4QHOA3##000100039X
3,A1MOSTXNIO5MPJ##000100039X
4,A2XQ5LZHTD4AFT##000100039X


In [72]:
reviews_df = pd.DataFrame(df4['reviewText'].progress_apply(lambda review: review.split("\n")[0]))
reviews_df.head()





Progress::   0%|          | 0/8898041 [00:00<?, ?it/s]



Progress::   0%|          | 1/8898041 [00:00<721:24:57,  3.43it/s]



Progress::   1%|          | 80853/8898041 [00:00<500:24:12,  4.89it/s]



Progress::   2%|▏         | 167018/8898041 [00:00<346:51:36,  6.99it/s]



Progress::   3%|▎         | 252609/8898041 [00:00<240:25:21,  9.99it/s]



Progress::   4%|▎         | 326488/8898041 [00:00<166:51:31, 14.27it/s]



Progress::   5%|▍         | 416101/8898041 [00:00<115:34:50, 20.38it/s]



Progress::   6%|▌         | 510341/8898041 [00:00<80:00:30, 29.12it/s] 



Progress::   7%|▋         | 601580/8898041 [00:00<55:23:50, 41.60it/s]



Progress::   8%|▊         | 694773/8898041 [00:01<38:20:35, 59.43it/s]



Progress::   9%|▉         | 786616/8898041 [00:01<26:32:26, 84.90it/s]



Progress::  10%|▉         | 873996/8898041 [00:01<18:22:44, 121.27it/s]



Progress::  11%|█         | 961202/8898041 [00:01<12:43:34, 173.24it/s]



Progress::  12%|█▏        | 1052898/8898041 [00

,reviewText
0,Spiritually and mentally inspiring! A book tha...
1,This is one my must have books. It is a master...
2,This book provides a reflection that you can a...
3,I first read THE PROPHET in college back in th...
4,A timeless classic. It is a very demanding an...


### Data Normalisation
<span style="color:red">**NOTICE:** As I am in a hurry to complete the first pipeline, I will only work with the first 100k reviews.</span>

* Tokenization <span style="color:blue"> DONE </span>
* Convert All Tokens to Lowercase <span style="color:blue"> DONE </span>
* Eliminate Punctuation <span style="color:blue"> DONE </span>
* Remove Stop Words <span style="color:blue"> DONE </span>
* Changing Numbers into Words <span style="color:blue"> DONE </span>
* Expand Abbreviations <span style="color:red"> NOT AS EASY AS I THOUGHT AND DOES NOT ADD MUCH VALUE (Ask Stasha's opinion)</span> 
* Correct Spelling <span style="color:red"> TOO SLOW (10h for 100k reviews)-->SO WONT DO</span>
* Substituting Tokens with Synonyms <span style="color:green"> TO DO</span>
* Semantical Marking of Negatives <span style="color:blue"> DONE (Ask Stasha's opinion) </span>

### Tokenization

In [162]:
step_0_df = reviews_df['reviewText'][0:99999].progress_apply(lambda review: nltk.word_tokenize(review))
step_0_df.head()










Progress::   0%|          | 0/99999 [00:00<?, ?it/s]








Progress::   0%|          | 152/99999 [00:00<01:05, 1516.74it/s]








Progress::   0%|          | 297/99999 [00:00<01:06, 1495.60it/s]








Progress::   0%|          | 445/99999 [00:00<01:07, 1485.35it/s]








Progress::   1%|          | 642/99999 [00:00<01:01, 1602.99it/s]








Progress::   1%|          | 799/99999 [00:00<01:02, 1588.20it/s]








Progress::   1%|          | 972/99999 [00:00<01:00, 1627.62it/s]








Progress::   1%|          | 1119/99999 [00:00<01:03, 1569.46it/s]








Progress::   1%|▏         | 1276/99999 [00:00<01:02, 1569.49it/s]








Progress::   1%|▏         | 1426/99999 [00:00<01:03, 1544.71it/s]








Progress::   2%|▏         | 1597/99999 [00:01<01:01, 1590.68it/s]








Progress::   2%|▏         | 1766/99999 [00:01<01:00, 1618.29it/s]








Progress::   2%|▏         | 1926/99999 [00:01<01:01, 1603.92it/s]








Progress::   2%|▏         | 2124/99999 [00:01<00

Progress::  11%|█         | 10827/99999 [00:08<01:24, 1054.42it/s]








Progress::  11%|█         | 10943/99999 [00:08<01:22, 1079.99it/s]








Progress::  11%|█         | 11055/99999 [00:08<01:21, 1090.30it/s]








Progress::  11%|█         | 11166/99999 [00:09<01:24, 1052.88it/s]








Progress::  11%|█▏        | 11288/99999 [00:09<01:20, 1097.17it/s]








Progress::  11%|█▏        | 11424/99999 [00:09<01:16, 1164.26it/s]








Progress::  12%|█▏        | 11543/99999 [00:09<01:18, 1120.61it/s]








Progress::  12%|█▏        | 11657/99999 [00:09<01:23, 1055.50it/s]








Progress::  12%|█▏        | 11787/99999 [00:09<01:19, 1116.16it/s]








Progress::  12%|█▏        | 11937/99999 [00:09<01:12, 1207.86it/s]








Progress::  12%|█▏        | 12062/99999 [00:09<01:15, 1170.20it/s]








Progress::  12%|█▏        | 12182/99999 [00:09<01:15, 1159.62it/s]








Progress::  12%|█▏        | 12305/99999 [00:09<01:14, 1179.80it/s]








Progress::  12%|█▏       

Progress::  21%|██        | 20784/99999 [00:17<01:05, 1202.63it/s]








Progress::  21%|██        | 20907/99999 [00:17<01:05, 1200.85it/s]








Progress::  21%|██        | 21029/99999 [00:17<01:10, 1117.10it/s]








Progress::  21%|██        | 21144/99999 [00:17<01:14, 1063.35it/s]








Progress::  21%|██▏       | 21268/99999 [00:17<01:11, 1093.61it/s]








Progress::  21%|██▏       | 21390/99999 [00:17<01:09, 1127.56it/s]








Progress::  22%|██▏       | 21505/99999 [00:17<01:09, 1125.32it/s]








Progress::  22%|██▏       | 21619/99999 [00:18<01:09, 1128.51it/s]








Progress::  22%|██▏       | 21734/99999 [00:18<01:09, 1133.36it/s]








Progress::  22%|██▏       | 21872/99999 [00:18<01:05, 1194.58it/s]








Progress::  22%|██▏       | 22032/99999 [00:18<01:00, 1292.64it/s]








Progress::  22%|██▏       | 22231/99999 [00:18<00:53, 1444.18it/s]








Progress::  22%|██▏       | 22384/99999 [00:18<00:58, 1334.91it/s]








Progress::  23%|██▎      

Progress::  33%|███▎      | 33022/99999 [00:26<00:56, 1189.81it/s]








Progress::  33%|███▎      | 33245/99999 [00:26<00:48, 1374.64it/s]








Progress::  33%|███▎      | 33422/99999 [00:26<00:45, 1468.32it/s]








Progress::  34%|███▎      | 33596/99999 [00:26<00:43, 1539.73it/s]








Progress::  34%|███▍      | 33811/99999 [00:26<00:39, 1682.83it/s]








Progress::  34%|███▍      | 33992/99999 [00:26<00:39, 1664.12it/s]








Progress::  34%|███▍      | 34200/99999 [00:26<00:37, 1769.53it/s]








Progress::  34%|███▍      | 34385/99999 [00:27<00:37, 1729.53it/s]








Progress::  35%|███▍      | 34574/99999 [00:27<00:36, 1773.28it/s]








Progress::  35%|███▍      | 34793/99999 [00:27<00:34, 1880.06it/s]








Progress::  35%|███▍      | 34986/99999 [00:27<00:35, 1847.65it/s]








Progress::  35%|███▌      | 35175/99999 [00:27<00:35, 1842.17it/s]








Progress::  35%|███▌      | 35371/99999 [00:27<00:34, 1874.65it/s]








Progress::  36%|███▌     

Progress::  45%|████▌     | 45271/99999 [00:35<00:51, 1065.26it/s]








Progress::  45%|████▌     | 45380/99999 [00:35<00:52, 1042.58it/s]








Progress::  45%|████▌     | 45486/99999 [00:35<00:53, 1011.82it/s]








Progress::  46%|████▌     | 45589/99999 [00:35<00:54, 995.45it/s] 








Progress::  46%|████▌     | 45695/99999 [00:35<00:53, 1013.69it/s]








Progress::  46%|████▌     | 45808/99999 [00:35<00:51, 1045.50it/s]








Progress::  46%|████▌     | 45919/99999 [00:35<00:50, 1063.55it/s]








Progress::  46%|████▌     | 46045/99999 [00:35<00:48, 1115.70it/s]








Progress::  46%|████▌     | 46158/99999 [00:36<00:52, 1031.66it/s]








Progress::  46%|████▋     | 46264/99999 [00:36<00:51, 1036.91it/s]








Progress::  46%|████▋     | 46370/99999 [00:36<00:57, 937.43it/s] 








Progress::  46%|████▋     | 46484/99999 [00:36<00:54, 990.04it/s]








Progress::  47%|████▋     | 46587/99999 [00:36<00:53, 998.34it/s]








Progress::  47%|████▋     |

...

Progress::  65%|██████▌   | 65088/99999 [00:53<00:24, 1417.87it/s]








Progress::  65%|██████▌   | 65249/99999 [00:53<00:23, 1470.36it/s]








Progress::  65%|██████▌   | 65400/99999 [00:53<00:23, 1481.75it/s]








Progress::  66%|██████▌   | 65550/99999 [00:53<00:24, 1387.08it/s]








Progress::  66%|██████▌   | 65692/99999 [00:53<00:26, 1306.17it/s]








Progress::  66%|██████▌   | 65826/99999 [00:53<00:29, 1146.40it/s]








Progress::  66%|██████▌   | 65947/99999 [00:53<00:30, 1104.35it/s]








Progress::  66%|██████▌   | 66062/99999 [00:53<00:32, 1049.49it/s]








Progress::  66%|██████▌   | 66171/99999 [00:54<00:37, 897.80it/s] 








Progress::  66%|██████▋   | 66275/99999 [00:54<00:36, 925.92it/s]








Progress::  66%|██████▋   | 66375/99999 [00:54<00:35, 944.81it/s]








Progress::  67%|██████▋   | 66511/99999 [00:54<00:32, 1038.49it/s]








Progress::  67%|██████▋   | 66622/99999 [00:54<00:31, 1047.41it/s]








Progress::  67%|██████▋   |

Progress::  74%|███████▍  | 74350/99999 [01:01<00:22, 1125.48it/s]








Progress::  74%|███████▍  | 74471/99999 [01:01<00:22, 1148.56it/s]








Progress::  75%|███████▍  | 74587/99999 [01:02<00:22, 1130.79it/s]








Progress::  75%|███████▍  | 74701/99999 [01:02<00:23, 1091.52it/s]








Progress::  75%|███████▍  | 74811/99999 [01:02<00:25, 979.68it/s] 








Progress::  75%|███████▍  | 74925/99999 [01:02<00:24, 1021.14it/s]








Progress::  75%|███████▌  | 75030/99999 [01:02<00:25, 970.64it/s] 








Progress::  75%|███████▌  | 75130/99999 [01:02<00:27, 894.07it/s]








Progress::  75%|███████▌  | 75246/99999 [01:02<00:25, 959.97it/s]








Progress::  75%|███████▌  | 75350/99999 [01:02<00:25, 978.61it/s]








Progress::  75%|███████▌  | 75455/99999 [01:02<00:24, 998.86it/s]








Progress::  76%|███████▌  | 75557/99999 [01:03<00:24, 1002.79it/s]








Progress::  76%|███████▌  | 75659/99999 [01:03<00:25, 965.61it/s] 








Progress::  76%|███████▌  | 7

Progress::  85%|████████▌ | 85242/99999 [01:10<00:14, 997.43it/s] 








Progress::  85%|████████▌ | 85344/99999 [01:10<00:14, 980.13it/s]








Progress::  85%|████████▌ | 85444/99999 [01:10<00:14, 976.53it/s]








Progress::  86%|████████▌ | 85543/99999 [01:10<00:15, 945.92it/s]








Progress::  86%|████████▌ | 85657/99999 [01:11<00:14, 993.22it/s]








Progress::  86%|████████▌ | 85758/99999 [01:11<00:14, 971.52it/s]








Progress::  86%|████████▌ | 85875/99999 [01:11<00:13, 1019.89it/s]








Progress::  86%|████████▌ | 85986/99999 [01:11<00:13, 1042.82it/s]








Progress::  86%|████████▌ | 86092/99999 [01:11<00:14, 973.16it/s] 








Progress::  86%|████████▌ | 86191/99999 [01:11<00:16, 829.22it/s]








Progress::  86%|████████▋ | 86322/99999 [01:11<00:14, 930.02it/s]








Progress::  86%|████████▋ | 86428/99999 [01:11<00:14, 964.12it/s]








Progress::  87%|████████▋ | 86558/99999 [01:11<00:12, 1044.66it/s]








Progress::  87%|████████▋ | 86680

Progress::  94%|█████████▍| 94464/99999 [01:19<00:06, 904.75it/s] 








Progress::  95%|█████████▍| 94568/99999 [01:19<00:05, 941.27it/s]








Progress::  95%|█████████▍| 94666/99999 [01:19<00:05, 945.79it/s]








Progress::  95%|█████████▍| 94786/99999 [01:19<00:05, 1009.81it/s]








Progress::  95%|█████████▍| 94905/99999 [01:19<00:04, 1056.93it/s]








Progress::  95%|█████████▌| 95040/99999 [01:20<00:04, 1127.63it/s]








Progress::  95%|█████████▌| 95233/99999 [01:20<00:03, 1287.67it/s]








Progress::  96%|█████████▌| 95506/99999 [01:20<00:02, 1529.88it/s]








Progress::  96%|█████████▌| 95733/99999 [01:20<00:02, 1694.31it/s]








Progress::  96%|█████████▌| 95927/99999 [01:20<00:03, 1326.20it/s]








Progress::  96%|█████████▌| 96089/99999 [01:20<00:03, 1229.24it/s]








Progress::  96%|█████████▌| 96234/99999 [01:20<00:02, 1255.91it/s]








Progress::  96%|█████████▋| 96401/99999 [01:20<00:02, 1350.24it/s]








Progress::  97%|█████████▋|

0    [Spiritually, and, mentally, inspiring, !, A, ...
1    [This, is, one, my, must, have, books, ., It, ...
2    [This, book, provides, a, reflection, that, yo...
3    [I, first, read, THE, PROPHET, in, college, ba...
4    [A, timeless, classic, ., It, is, a, very, dem...
Name: reviewText, dtype: object

### Convert Tokens to Lowercase

In [168]:
import re
import string

def convert_to_lowercase(review):

    for i in range(len(review)):
        review[i] = review[i].lower()
    return review

In [169]:
step_1_df = step_0_df.progress_apply(lambda review: convert_to_lowercase(review))
step_1_df.head()










Progress::   0%|          | 0/99999 [00:00<?, ?it/s]








Progress::   6%|▌         | 5777/99999 [00:00<00:01, 57767.16it/s]








Progress::  10%|█         | 10307/99999 [00:00<00:01, 53357.27it/s]








Progress::  15%|█▍        | 14998/99999 [00:00<00:01, 51234.98it/s]








Progress::  20%|█▉        | 19881/99999 [00:00<00:01, 50485.85it/s]








Progress::  25%|██▌       | 25180/99999 [00:00<00:01, 51209.21it/s]








Progress::  32%|███▏      | 31672/99999 [00:00<00:01, 54667.15it/s]








Progress::  38%|███▊      | 38400/99999 [00:00<00:01, 57923.38it/s]








Progress::  44%|████▍     | 43833/99999 [00:00<00:01, 52240.89it/s]








Progress::  49%|████▉     | 48911/99999 [00:00<00:01, 45617.63it/s]








Progress::  54%|█████▎    | 53522/99999 [00:01<00:01, 41882.44it/s]








Progress::  58%|█████▊    | 57805/99999 [00:01<00:01, 40427.31it/s]








Progress::  62%|██████▏   | 61925/99999 [00:01<00:00, 38496.11it/s]








Progress::  66%|███

0    [spiritually, and, mentally, inspiring, !, a, ...
1    [this, is, one, my, must, have, books, ., it, ...
2    [this, book, provides, a, reflection, that, yo...
3    [i, first, read, the, prophet, in, college, ba...
4    [a, timeless, classic, ., it, is, a, very, dem...
Name: reviewText, dtype: object

### Eliminate Punctuation

In [170]:
import re
import string

def eliminate_punctuation(review, regex):
    new_review = []
    for token in review:
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    return new_review

In [171]:
regex=re.compile('[%s]' % re.escape(string.punctuation))

step_2_df = step_1_df.progress_apply(lambda review: eliminate_punctuation(review, regex))
step_2_df.head()










Progress::   0%|          | 0/99999 [00:00<?, ?it/s]








Progress::   2%|▏         | 1752/99999 [00:00<00:05, 17023.91it/s]








Progress::   3%|▎         | 3397/99999 [00:00<00:05, 16845.81it/s]








Progress::   5%|▍         | 4729/99999 [00:00<00:06, 15601.63it/s]








Progress::   7%|▋         | 6501/99999 [00:00<00:05, 16181.04it/s]








Progress::   8%|▊         | 8002/99999 [00:00<00:05, 15809.19it/s]








Progress::  10%|▉         | 9593/99999 [00:00<00:05, 15836.69it/s]








Progress::  11%|█         | 11071/99999 [00:00<00:05, 15497.37it/s]








Progress::  12%|█▏        | 12490/99999 [00:00<00:05, 14854.41it/s]








Progress::  14%|█▍        | 13890/99999 [00:00<00:06, 13627.71it/s]








Progress::  15%|█▌        | 15290/99999 [00:01<00:06, 13733.26it/s]








Progress::  17%|█▋        | 16636/99999 [00:01<00:06, 13560.48it/s]








Progress::  18%|█▊        | 18219/99999 [00:01<00:05, 14169.90it/s]








Progress::  20%|█▉      

0    [spiritually, and, mentally, inspiring, a, boo...
1    [this, is, one, my, must, have, books, it, is,...
2    [this, book, provides, a, reflection, that, yo...
3    [i, first, read, the, prophet, in, college, ba...
4    [a, timeless, classic, it, is, a, very, demand...
Name: reviewText, dtype: object

In [172]:
### Remove Stop Words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(review):
    return [token for token in review if not token in stop_words]

step_3_df = step_2_df.progress_apply(lambda review: numstringToWord(review, p))
step_3_df.head()










Progress::   0%|          | 0/99999 [00:00<?, ?it/s]








Progress::   5%|▍         | 4955/99999 [00:00<00:01, 49522.65it/s]








Progress::   9%|▉         | 8929/99999 [00:00<00:01, 46106.34it/s]








Progress::  13%|█▎        | 13029/99999 [00:00<00:01, 44441.81it/s]








Progress::  17%|█▋        | 16522/99999 [00:00<00:02, 41084.53it/s]








Progress::  21%|██        | 21032/99999 [00:00<00:01, 42208.11it/s]








Progress::  24%|██▍       | 24486/99999 [00:00<00:02, 36142.63it/s]








Progress::  28%|██▊       | 27716/99999 [00:00<00:02, 33253.37it/s]








Progress::  32%|███▏      | 32408/99999 [00:00<00:01, 36435.77it/s]








Progress::  38%|███▊      | 37806/99999 [00:00<00:01, 40368.75it/s]








Progress::  42%|████▏     | 41930/99999 [00:01<00:01, 38920.23it/s]








Progress::  46%|████▌     | 45894/99999 [00:01<00:01, 37022.46it/s]








Progress::  50%|████▉     | 49777/99999 [00:01<00:01, 37540.04it/s]








Progress::  54%|████

0    [spiritually, and, mentally, inspiring, a, boo...
1    [this, is, one, my, must, have, books, it, is,...
2    [this, book, provides, a, reflection, that, yo...
3    [i, first, read, the, prophet, in, college, ba...
4    [a, timeless, classic, it, is, a, very, demand...
Name: reviewText, dtype: object

### Changing Numbers into Words

In [174]:
import inflect
p = inflect.engine()

def numstringToWord(review, p):

    for i in range(len(review)):
        if(review[i].isdigit()):
            review[i] = p.number_to_words(review[i])
    return review

In [175]:
step_4_df = step_3_df.progress_apply(lambda review: numstringToWord(review, p))
step_4_df.head()










Progress::   0%|          | 0/99999 [00:00<?, ?it/s]








Progress::   8%|▊         | 8428/99999 [00:00<00:01, 84276.66it/s]








Progress::  16%|█▌        | 16010/99999 [00:00<00:01, 81546.96it/s]








Progress::  25%|██▍       | 24529/99999 [00:00<00:00, 82606.70it/s]








Progress::  35%|███▍      | 34889/99999 [00:00<00:00, 87950.60it/s]








Progress::  45%|████▍     | 44567/99999 [00:00<00:00, 90423.23it/s]








Progress::  52%|█████▏    | 52285/99999 [00:00<00:00, 85976.03it/s]








Progress::  60%|██████    | 60270/99999 [00:00<00:00, 84037.21it/s]








Progress::  68%|██████▊   | 68491/99999 [00:00<00:00, 83472.90it/s]








Progress::  76%|███████▋  | 76415/99999 [00:00<00:00, 80328.88it/s]








Progress::  86%|████████▌ | 85889/99999 [00:01<00:00, 84166.30it/s]








Progress::  94%|█████████▍| 94164/99999 [00:01<00:00, 81818.11it/s]








Progress:: 100%|██████████| 99999/99999 [00:01<00:00, 84286.34it/s]

0    [spiritually, and, mentally, inspiring, a, boo...
1    [this, is, one, my, must, have, books, it, is,...
2    [this, book, provides, a, reflection, that, yo...
3    [i, first, read, the, prophet, in, college, ba...
4    [a, timeless, classic, it, is, a, very, demand...
Name: reviewText, dtype: object

### Correct Spelling

In [160]:
# from autocorrect import spell

# def spellCheck(review):

#     for i in range(len(review)):
#         review[i] = spell(review[i])
#     return review

In [161]:
# step_5_df = step_4_df.progress_apply(lambda review: spellCheck(review))
# step_5_df.head()









Progress::   0%|          | 0/99999 [00:00<?, ?it/s]







Progress::   0%|          | 5/99999 [00:00<1:50:40, 15.06it/s]







Progress::   0%|          | 6/99999 [00:00<5:59:42,  4.63it/s]







Progress::   0%|          | 9/99999 [00:01<5:19:03,  5.22it/s]







Progress::   0%|          | 11/99999 [00:04<15:26:57,  1.80it/s]







Progress::   0%|          | 15/99999 [00:04<12:04:48,  2.30it/s]







Progress::   0%|          | 16/99999 [00:05<12:27:17,  2.23it/s]







Progress::   0%|          | 17/99999 [00:07<27:16:02,  1.02it/s]







Progress::   0%|          | 18/99999 [00:17<99:11:09,  3.57s/it]







Progress::   0%|          | 19/99999 [00:17<70:24:21,  2.54s/it]







Progress::   0%|          | 23/99999 [00:17<49:42:16,  1.79s/it]







Progress::   0%|          | 24/99999 [00:17<36:18:39,  1.31s/it]







Progress::   0%|          | 31/99999 [00:17<25:43:45,  1.08it/s]







Progress::   0%|          | 35/99999 [00:18<18:51:06,  1.47it/s]







Pr

Progress::   0%|          | 163/99999 [03:03<28:30:53,  1.03s/it]







Progress::   0%|          | 164/99999 [03:03<23:06:52,  1.20it/s]







Progress::   0%|          | 169/99999 [03:04<16:52:41,  1.64it/s]







Progress::   0%|          | 173/99999 [03:05<13:51:59,  2.00it/s]







Progress::   0%|          | 175/99999 [03:05<12:12:59,  2.27it/s]







Progress::   0%|          | 178/99999 [03:06<8:55:11,  3.11it/s] 







Progress::   0%|          | 179/99999 [03:06<9:30:27,  2.92it/s]







Progress::   0%|          | 182/99999 [03:07<9:45:49,  2.84it/s]







Progress::   0%|          | 183/99999 [03:08<10:29:31,  2.64it/s]







Progress::   0%|          | 185/99999 [03:08<8:07:37,  3.41it/s] 







Progress::   0%|          | 187/99999 [03:09<9:03:15,  3.06it/s]







Progress::   0%|          | 188/99999 [03:09<10:32:07,  2.63it/s]







Progress::   0%|          | 189/99999 [03:11<20:40:32,  1.34it/s]







Progress::   0%|          | 191/99999 [03:27<81:37:39,

KeyboardInterrupt: 

### Substituting Tokens with Synonyms

In [ ]:
#TODO

### Semantical Marking of Negatives

In [176]:
def negation_tokenizer(review):

    # regex to match negation tokens
    negation_re = re.compile("""(?x)(?:^(?:never|no|nothing|nowhere|noone|none|not|havent|
            hasnt|hadnt|cant|couldnt|shouldnt|wont|wouldnt|dont|
            doesnt|didnt|isnt|arent|aint)$)|n't""")

    alter_re = re.compile("""(?x)(?:^(?:but|however|nevertheless|still|though|tho|yet)$)""")

    neg_review_tokens = []
    append_neg = False  # stores whether to add "_NEG"

    for token in review:

        # If append_neg is False
        if append_neg == False:

            # Check if the current token is a negation
            if negation_re.match(token):
                append_neg = True

        # but if a negation has been previously identified, check if this is an  alteration
        elif alter_re.match(token):
            append_neg = False

        # or if another negation appears
        elif negation_re.match(token):
            append_neg = False

        # and if not then append the suffix
        else:
            token += "_NEG"

        # append the new token in the return list
        neg_review_tokens.append(token)

    return neg_review_tokens

In [177]:
step_5_df = step_4_df.progress_apply(lambda review: negation_tokenizer(review))
step_5_df.head()










Progress::   0%|          | 0/99999 [00:00<?, ?it/s]








Progress::   2%|▏         | 1789/99999 [00:00<00:05, 17863.87it/s]








Progress::   3%|▎         | 3414/99999 [00:00<00:05, 17345.76it/s]








Progress::   5%|▍         | 4571/99999 [00:00<00:06, 14462.35it/s]








Progress::   6%|▌         | 5993/99999 [00:00<00:06, 14388.35it/s]








Progress::   7%|▋         | 7344/99999 [00:00<00:06, 14104.49it/s]








Progress::   9%|▊         | 8692/99999 [00:00<00:06, 13906.21it/s]








Progress::  10%|▉         | 9903/99999 [00:00<00:06, 13276.56it/s]








Progress::  11%|█         | 11111/99999 [00:00<00:06, 12841.24it/s]








Progress::  13%|█▎        | 12509/99999 [00:00<00:06, 13162.45it/s]








Progress::  14%|█▍        | 13771/99999 [00:01<00:07, 12309.37it/s]








Progress::  15%|█▍        | 14975/99999 [00:01<00:06, 12221.60it/s]








Progress::  16%|█▌        | 16179/99999 [00:01<00:07, 11924.14it/s]








Progress::  17%|█▋       

0    [spiritually, and, mentally, inspiring, a, boo...
1    [this, is, one, my, must, have, books, it, is,...
2    [this, book, provides, a, reflection, that, yo...
3    [i, first, read, the, prophet, in, college, ba...
4    [a, timeless, classic, it, is, a, very, demand...
Name: reviewText, dtype: object

In [180]:
step_5_df[1000]

['what',
 'a',
 'delightful',
 'read',
 'water',
 'for',
 'elephants',
 'has',
 'got',
 'to',
 'be',
 'one',
 'of',
 'the',
 'best',
 'reads',
 'for',
 'anyone',
 'that',
 'likes',
 'animals',
 'circuses',
 'and',
 'a',
 'very',
 'wonderfully',
 'flowing',
 'story',
 'working',
 'in',
 'or',
 'for',
 'a',
 'circus',
 'especially',
 'in',
 'past',
 'years',
 'had',
 'to',
 'be',
 'one',
 'of',
 'the',
 'most',
 'grueling',
 'and',
 'tough',
 'jobs',
 'to',
 'tackle',
 'sara',
 'gruen',
 'makes',
 'the',
 'reader',
 'smell',
 'the',
 'circus',
 'smells',
 'taste',
 'the',
 'midway',
 'foods',
 'and',
 'be',
 'in',
 'the',
 'ring',
 'with',
 'the',
 'animals',
 'as',
 'they',
 'entertain',
 'i',
 'have',
 'not',
 'been_NEG',
 'to_NEG',
 'a_NEG',
 'circus_NEG',
 'since_NEG',
 'i_NEG',
 'was_NEG',
 'a_NEG',
 'teenager_NEG',
 'which_NEG',
 'is_NEG',
 'many_NEG',
 'years_NEG',
 'ago_NEG',
 'water_NEG',
 'for_NEG',
 'elephants_NEG',
 'took_NEG',
 'me_NEG',
 'back_NEG',
 'to_NEG',
 'those_NEG',